# QC-LLM Tutorial: Computing Quantum Coherence in Text

This notebook demonstrates how to use the QC-LLM library to evaluate quantum coherence in language model outputs.

## Overview

The QC-LLM library implements the **f₀ = 141.7001 Hz** quantum coherence standard for evaluating semantic coherence in text. It uses:

1. **BERT Embeddings**: Semantic vector representations of text
2. **FFT Analysis**: Spectral decomposition to find dominant frequencies  
3. **f₀ Alignment**: Measures how well text aligns with the fundamental frequency

## Installation

```bash
pip install transformers>=4.48.0 torch>=2.6.0 numpy scipy
```

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '../../API/Python')

from qc_llm import compute_coherence, QC_LLM, F0
import numpy as np
import matplotlib.pyplot as plt

print(f"Fundamental Frequency f₀ = {F0} Hz")

## Example 1: Basic Coherence Analysis

Let's analyze a simple text for quantum coherence.

In [ ]:
# Sample text to analyze
text = "Quantum coherence is a fascinating property that emerges in complex systems."

# Compute coherence (without BERT for faster execution)
result = compute_coherence(text, use_bert=False)

print("Coherence Analysis Results:")
print("="*50)
print(f"Coherence Score: {result['coherence']:.3f}")
print(f"Frequency Alignment: {result['frequency_alignment']:.3f}")
print(f"Quantum Metric: {result['quantum_metric']:.3f}")
print(f"Recommendation: {result['recommendation']}")

## Example 2: Comparing Different Texts

Let's compare coherence scores across different text samples.

In [ ]:
# Test samples with varying complexity
samples = [
    "The quantum field exhibits remarkable coherence properties across multiple scales.",
    "Hello world",
    "quantum quantum quantum quantum",
    "The integration of quantum mechanics with general relativity remains an open challenge in theoretical physics.",
    "a b c d e f g h i j k l m n o p"
]

print("Comparative Coherence Analysis:")
print("="*80)

results = []
for i, text in enumerate(samples, 1):
    result = compute_coherence(text, use_bert=False)
    results.append(result)
    
    print(f"\nSample {i}: {text[:60]}{'...' if len(text) > 60 else ''}")
    print(f"  Coherence: {result['coherence']:.3f} | {result['recommendation']}")
    print(f"  Freq Align: {result['frequency_alignment']:.3f} | Quantum: {result['quantum_metric']:.3f}")

## Example 3: Visualizing Coherence Metrics

In [ ]:
# Extract metrics for plotting
coherences = [r['coherence'] for r in results]
freq_aligns = [r['frequency_alignment'] for r in results]
quantum_metrics = [r['quantum_metric'] for r in results]

# Create bar plot
x = np.arange(len(samples))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(x - width, coherences, width, label='Coherence', alpha=0.8)
ax.bar(x, freq_aligns, width, label='Frequency Alignment', alpha=0.8)
ax.bar(x + width, quantum_metrics, width, label='Quantum Metric', alpha=0.8)

ax.set_xlabel('Sample')
ax.set_ylabel('Score')
ax.set_title('Quantum Coherence Metrics Comparison')
ax.set_xticks(x)
ax.set_xticklabels([f'S{i+1}' for i in range(len(samples))])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add threshold lines
ax.axhline(y=0.8, color='g', linestyle='--', alpha=0.5, label='High Threshold')
ax.axhline(y=0.6, color='y', linestyle='--', alpha=0.5, label='Moderate Threshold')

plt.tight_layout()
plt.show()

## Example 4: Using the QC_LLM Class

The QC_LLM class provides a convenient wrapper for coherence analysis.

In [ ]:
# Initialize QC_LLM validator
validator = QC_LLM(model_name="gpt-4-sim")

# Single validation
text = "The emergence of quantum coherence in biological systems suggests deep connections."
result = validator.validate(text)

print("QC_LLM Validation:")
print(f"Text: {text}")
print(f"Coherence: {result['coherence']:.2%}")
print(f"Recommendation: {result['recommendation']}")

## Example 5: Batch Processing

Process multiple texts efficiently with batch validation.

In [ ]:
# Batch of texts to validate
texts = [
    "Quantum coherence enables information processing.",
    "The sky is blue and clouds are white.",
    "Neural networks exhibit emergent properties."
]

# Batch validation
results = validator.batch_validate(texts)

print("Batch Validation Results:")
print("="*80)
for i, (text, result) in enumerate(zip(texts, results), 1):
    print(f"\n{i}. {text}")
    print(f"   Coherence: {result['coherence']:.2%} - {result['recommendation']}")

## Example 6: Understanding Quantum Metric

The quantum metric measures token repetition:
- **High value (→1)**: Very repetitive text
- **Low value (→0)**: Highly diverse vocabulary

In [ ]:
# Repetitive vs. diverse text
repetitive = "quantum quantum quantum quantum quantum"
diverse = "quantum coherence frequency alignment spectral analysis"

result_rep = compute_coherence(repetitive, use_bert=False)
result_div = compute_coherence(diverse, use_bert=False)

print("Quantum Metric Comparison:")
print("="*60)
print(f"Repetitive text: '{repetitive}'")
print(f"  Quantum Metric: {result_rep['quantum_metric']:.3f} (high = repetitive)")
print(f"\nDiverse text: '{diverse}'")
print(f"  Quantum Metric: {result_div['quantum_metric']:.3f} (low = diverse)")

# Visualize
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Plot 1: Quantum metrics
ax[0].bar(['Repetitive', 'Diverse'], 
          [result_rep['quantum_metric'], result_div['quantum_metric']], 
          color=['red', 'green'], alpha=0.7)
ax[0].set_ylabel('Quantum Metric')
ax[0].set_title('Token Repetition Analysis')
ax[0].set_ylim([0, 1])
ax[0].grid(axis='y', alpha=0.3)

# Plot 2: Overall coherence
ax[1].bar(['Repetitive', 'Diverse'],
          [result_rep['coherence'], result_div['coherence']],
          color=['red', 'green'], alpha=0.7)
ax[1].set_ylabel('Coherence Score')
ax[1].set_title('Overall Coherence')
ax[1].set_ylim([0, 1])
ax[1].axhline(y=0.6, color='orange', linestyle='--', alpha=0.5)
ax[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Example 7: Effect of Text Length

In [ ]:
# Generate texts of varying length
base_text = "Quantum coherence emerges naturally in complex systems. "
lengths = [1, 2, 5, 10, 20]

length_results = []
for n in lengths:
    text = (base_text * n).strip()
    result = compute_coherence(text, use_bert=False)
    length_results.append(result['coherence'])

# Plot
plt.figure(figsize=(10, 5))
plt.plot([len(base_text) * n for n in lengths], length_results, 'o-', linewidth=2, markersize=8)
plt.xlabel('Text Length (characters)')
plt.ylabel('Coherence Score')
plt.title('Effect of Text Length on Coherence')
plt.grid(alpha=0.3)
plt.axhline(y=0.6, color='orange', linestyle='--', alpha=0.5, label='Moderate Threshold')
plt.legend()
plt.show()

print(f"\nCoherence scores by length:")
for n, score in zip(lengths, length_results):
    print(f"  {len(base_text) * n} chars: {score:.3f}")

## Example 8: BERT-Based Analysis (Advanced)

**Note**: This requires transformers library and may take longer to execute.

```python
# Uncomment to run with BERT
# text = "The quantum field exhibits remarkable coherence properties."
# result_bert = compute_coherence(text, use_bert=True)
# 
# print(f"BERT-based coherence: {result_bert['coherence']:.3f}")
# print(f"Recommendation: {result_bert['recommendation']}")
```

## Summary

You've learned how to:
1. ✅ Compute quantum coherence scores
2. ✅ Compare different texts
3. ✅ Visualize coherence metrics
4. ✅ Use the QC_LLM class
5. ✅ Batch process texts
6. ✅ Understand quantum metrics
7. ✅ Analyze text length effects

## Next Steps

- Read [Mathematical Foundation](../../Documentation/Theory/mathematical_foundation.md)
- Explore [LLM Integration Examples](../LLM_Integration/)
- Check [API Documentation](../../API/README.md)

## References

- Paper: [DOI: 10.5281/zenodo.17379721](https://doi.org/10.5281/zenodo.17379721)
- Repository: [github.com/motanova84/141hz](https://github.com/motanova84/141hz)
